# Herramientas de apoyo para el análisis y procesado de datos del secuenciador de ADN MinION

**Autor:** Orlandy Ariel Sánchez Acosta.

**Correo:** alu0100773408@ull.edu.es

**Asignatura:** Trabajo de Fin de Grado.

# Información del TFG
Se realizará un proyecto de bioinformática que tiene como objetivo realizar el análisis y procesado de datos para detectar patrones de posibles fallos en las lecturas del secuenciador MinION. Esto será de gran utilidad de cara al ensamblaje del ADN, dado que quitando los fallos este proceso será más óptimo.

El _Dr. Carlos Flores_, quien proporciona los datos, será quien validará que los procesos y métodos son los correctos. Estos datos posteriormente serán ensamblados para la manipulación de información relativa a los datos biológicos, tales como macromoléculas (por ejemplo, ADN o proteínas).

El proceso consistirá en realizar una serie de filtros por los cuales pasar el fichero que se pretenda analizar. De cada uno de los filtros podremos obtener salidas de tipo graficas o texto, inicialmente en formato FASTQ. Las salidas las diferenciaremos en salidas buenas y malas.

Una vez se pasen los filtros se procederá a buscar cuáles lecturas son las más malas de entre las malas, es decir, se buscarán coincidencias dentro de las lecturas malas que se han obtenido de los distintos filtros.

Como se mencionó anteriormente se utilizarán distintos filtros, donde tendremos los siguientes filtros:
* Detección de las dos letras más repetidas.
* Contenido GC.
* Calidades.

![Preprocesado](https://raw.githubusercontent.com/OrlandyAriel/Herramienta_de_apoyo_para_el_an-lisis_y_procesado_de_datos_del_secuenciador_de_ADN_MinION/master/Imagenes/esquema.jpg)
> ** MiniION **

> MiniION es un dispositivo de fabricado por Oxfored Nanopore, tiene como fin secuenciar ADN, que comparado con los distintos dispositivos que hay en el mercado, este tiene un bajo costo. Además, es un aparato relativamente pequeño por lo que hace que sea de gran utilidad en el campo de la genética.

A continuación, se realizará el proceso del análisis de los reads. 

## Parámetros configurables
* **FICHERO_ENTRADA:** < Nombre del fichero que contiene los reads >

* **FICHERO_SALIDA_MALAS:**< Nombre del fichero donde se guardarán los fallos >

* **FICHERO_SALIDA_BUENAS:** < Nombre del fichero donde se guardarán las lecturas no fallidas >

* **FORMATO:** < Nombre del formato que se pretender leer >

* **UMBRAL:** < Umbral con el que detectar los fallos >

* **D_PORC_ATIPIOS:** < Esta variable será el porcentaje que se quiere quitar de datos atípicos >

* **IT_CONFIANZA:** < Esta variable será la utilizada para calcular el valor intervalo de confianza >

* **PATH_SALIDA_DOS_MAS_REP:** < Ruta del directorio para los ficheros de salida del método de las dos letras más repetidas >

* **PATH_SALIDA_GC:** < Ruta del directorio para los ficheros de salida del método de Contenido GC >

* **PATH_SALIDA_CALIDAD:** < Ruta del directorio para los ficheros de salida del método de las Calidades >

> **NOTA 1:** Para mayor comodidad el fichero con el que se trabajará está en el mismo directorio que a su vez estará dentro de la subcarperta DatosOriginales que el cuaderno.
>
> **NOTA 2:** El _FORMATO_ debe coincidir con el que tiene el fichero, de lo contrario no se leerá correctamente el fichero pudiendo ocasionar errores.

In [1]:
FICHERO_ENTRADA = "DatosOriginales/secuencias_originales.fastq" #FICHERO original
FORMATO = "fastq"

In [2]:
D_PORC_ATIPICOS = 5 #Esta variable será el porcentaje que se quiere quitar de datos atípicos
IT_CONFIANZA = 0.95 # Esta variable será la utilizada para calcular el intervalo de confianza.
UMBRAL = 80 #umbral para la detección de fallos

In [3]:
FICHERO_SALIDA_MALAS = "Lecturas_malas.fastq"
FICHERO_SALIDA_BUENAS = "Lecturas_buenas.fastq"

In [4]:
PATH_SALIDA_DOS_MAS_REP = "Salidas/DosMasRepetidas/"
PATH_SALIDA_GC = "Salidas/ContenidoGC/"
PATH_SALIDA_CALIDAD = "Salidas/Calidades/"
PATH_SALIDA_COIN = "Salidas/Coincidencia/"

In [5]:
FICHERO_ENTRADA_BLAST = "Salidas/Coincidencia/Lecturas_malas.fastq"
FORMATO_ENTRADA_BLAST = "fastq"
FICHERO_SALIDA_BLAST = "Salidas/Coincidencia/Lecturas_malas.fasta"
FORMATO_SALIDA_BLAST = "fasta"
#####
PROGRAMA_BLAST = "blastn"
BASE_DE_DATOS_BLAST = "nr"

## Librerías 
En caso de dudas o cualquier consulta sobre las librerías usadas, consultar los siguientes enlaces.
* [Biopython](http://biopython.org/wiki/Documentation)
* [Plotly](https://plot.ly/python/)
* [Iteratools](https://docs.python.org/2/library/itertools.html)
* [IPython](https://ipython.readthedocs.io/en/stable/install/install.html)
* [Pandas](http://pandas.pydata.org/)
* [NumPy](http://www.numpy.org/)
* [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [URLLib](https://docs.python.org/3/howto/urllib2.html)
* [Peticiones BLAST](https://github.com/OrlandyAriel/BusquedaBLAST)
* [Time](https://docs.python.org/2/library/time.html)

A continuación, se pasará a importar todas las librerías/módulos que se utilizarán.


In [6]:
### Librería de BioInformática.
from Bio import SeqIO
from Bio.SeqUtils import GC
### Librería para realizar los gráficos 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.plotly as py
import plotly.graph_objs as go
### Librerías de utilidades
import time
import numpy as np
from IPython.display import Markdown,display, HTML
import itertools # libreria para combinatoria
import operator
import pandas as pd
from scipy.stats import norm
### Librerías propias
import GraficoDePuntos as gp
import Utilidades as utile
import UtilidadesParaArrays as arrUtile
import PresentacionDatos as psd
import putblast as put
import getblast as get

Calculo del Intervalo de confianza

In [7]:
IT_CONFIANZA = norm.ppf(IT_CONFIANZA)

## Lecturas de secuencias desde el fichero de entrada

Carga las lecturas del fichero en un array, de manera que solo lea una vez del fichero.

In [8]:
fichero_de_secuencias = [rec for rec in SeqIO.parse(FICHERO_ENTRADA, FORMATO)]

## Definición de las variables que se utilizarán para el análisis.

In [9]:
numero_de_A = numero_de_T = numero_de_C = numero_de_G = None
c_externa_utile = utile.Utilidades() #llamada a una clase propia para realizar el porcentaje y las permutaciones.
c_arrays_utile = arrUtile.UtilidadesParaArrays()
c_psd = psd.PresentacionDatos()
peticion_put = put.PeticionPUTBlast()
peticion_get = get.PeticionGETBlast()

dic_posibles_fallos = {} #Diccionario que guarda, en forma de clave:valor, la posicion:[array] de las lecturas fallidas.
dic_lec_buenas = {} #diccionario que guarda, de forma clave:valor, la posición:tamaño de la lectura correctas.
dic_lec_malas = {}#diccionario que guarda, de forma clave:valor, la posición:tamaño de la lectura fallidas.
dic_cal_buenas = {}#diccionario que guarda, de forma clave:valor, la posición:suma de la calidad de la secuencia buena
dic_cal_malas = {}#diccionario que guarda, de forma clave:valor, la posición:suma de la calidad de la secuencia mala.
dic_contenido_gc = {}#diccionario que guarda, de forma clave:valor, la posición:contenido GC de la secuencia.
dic_gc_buenos = {}# diccionario que gaurda, de forma clave:valor, la posición:contenidoGC de las secuencias buenas
dic_gc_malos = {}# diccionario que gaurda, de forma clave:valor, la posición:contenidoGC de las secuencias malas.
dic_rids = {} # diccionario donde se guardarán los rid's obtenidos de las busquedas en blast

array_fallos_lectura = [] # array temporal para guardar las lecturas fallidas para luego guardarlas en un fichero. 
array_buenos_lectura = [] # array temporal para guardar las lecturas buenas para luego guardarlas en un fichero.
suma_calidades = []# guarda la suma de todas la calidades de las secuencias del fichero.
array_gc_valores = [] # guarda en un array, ordenado de menor a mayor, el contenido en GC de cada secuencia.
array_resultados_blast = []


# Estadísticas Generales del fichero
* ** Lecturas**: Total de lecturas que contiene el FICHERO.
* ** Mínimo**: Tamaño de la secuencia menor en el FICHERO.
* ** Máximo**: Tamaño de la secuencia mayor en el FICHERO.

In [10]:
lecturas = [len(rec) for rec in fichero_de_secuencias]

In [11]:
tamanio = len(lecturas)#total de lecturas del fichero
minimo = min(lecturas)#lectrua más pequeña
maximo = max(lecturas)#lectura más grande
c_psd.printmd("* **El fichero: **%s"%FICHERO_ENTRADA)
c_psd.printmd("* **Total de READS (lecturas):** %i"%tamanio)
c_psd.printmd("* **Los READS(lecturas) tiene un tamaño de entre: **%i y %i"%(minimo,maximo))

* **El fichero: **DatosOriginales/secuencias_originales.fastq

* **Total de READS (lecturas):** 1171

* **Los READS(lecturas) tiene un tamaño de entre: **47 y 185942

# Filtro de detección de las dos letras más repetidas.
Para poder analizar las secuencias y sacar los posibles errores se hará lo siguiente:
1. Se calculará el porcentaje de cada una de las letras dentro de la secuencia, para ello:
    1. Se calcula el tamaño de la secuencia.
    2. Se cuentan las veces que aparece la letra en la secuencia
    3. Se llama a la función **tanto_por_ciento** y se le pasa por parámetro el número de veces que aparece la letra y el tamaño de la secuencia.
2. Se guardan tanto la letra como el porcentaje dentro de un diccionario. (que estará desordenado)
3. Se ordena el diccionario por el porcentaje, de menor a mayor.
4. Se suma el valor de las dos letras con mayor porcentaje y se comprueba si ese valor es mayor que el UMBRAL
    1. Si es mayor:
        1. Se guardará dentro del diccionario **dic_posible_fallo** la posición de la secuencia como clave y las combinaciones de las letras que más se repiten como valor.
        2. Se guardará dentro del diccionario **dic_lec_malas** la posición de la secuencia como clave y el tamaño de la secuencia como valor.
        3. Se guardará en el array **array_fallos_lectura** la secuencia.
    2. Si no es mayor:
        1. Se guardará en el diccionario **dic_lec_buenas** la posición de la secuencia como clave y el tamaño de la secuencia como valor. 
        4. Se guardará en el array **array_buenos_lectura** la secuencia
        
        
>**Nota:** este método fue descartado puesto que es muy arbitrario.

In [12]:
#esto dividirlo, las permutaciones como otro método aparte
for i, secuencia in enumerate(fichero_de_secuencias):
    size = len(secuencia)
    #porcentaje letras
    numero_de_A = c_externa_utile.tanto_por_ciento(secuencia.seq.count("A"),size)
    numero_de_T = c_externa_utile.tanto_por_ciento(secuencia.seq.count("T"),size)
    numero_de_C = c_externa_utile.tanto_por_ciento(secuencia.seq.count("C"),size)
    numero_de_G = c_externa_utile.tanto_por_ciento(secuencia.seq.count("G"),size)
    
    dic_porcentaje_desc = {'A':numero_de_A, 
                           'T':numero_de_T, 
                           'C':numero_de_C, 
                           'G':numero_de_G}
    dic_porcentaje_order = sorted(dic_porcentaje_desc.items(), key=operator.itemgetter(1))
    #si supera el UMBRAL, guarda en el diccionario correspodiente.
    if(dic_porcentaje_order[3][1] + dic_porcentaje_order[2][1]) > UMBRAL: 
        dic_posibles_fallos[i] = c_externa_utile.permutaciones(dic_porcentaje_order[3][0] + dic_porcentaje_order[2][0])
        dic_lec_malas[i] = size
        array_fallos_lectura.append(secuencia)
    else:
        dic_lec_buenas[i] = size
        array_buenos_lectura.append(secuencia)

## Preparación de los datos para representarlos
Se extraen los datos de los diccionarios dic_lec_buenas y dic_lec_malas para poder representarlos en las gráficas, para ello se guardarán en los arrays **fallos_x, fallos_y, correctos_x** y **correctos_y**

In [13]:
fallos_x = c_arrays_utile.obtener_claves_dic(dic_lec_malas)
fallos_y = c_arrays_utile.obtener_claves_dic(dic_lec_malas,clave=False)

In [14]:
correctos_x = c_arrays_utile.obtener_claves_dic(dic_lec_buenas)
correctos_y = c_arrays_utile.obtener_claves_dic(dic_lec_buenas,clave=False)

# Gráfica de las detecciones de las dos letras más repetidas
Gráfico de barras donde se muestran las lecturas con su tamaño y su número de secuencia, en la gráfica se muestran tanto las lecturas buenas como las lecturas malas.

>**Nota:** En la leyenda se puede activar lo que se quiere ver, es decir, si sólo se quiere ver las lecturas malas pulsar, para desactivar, la leyenda de las lecturas buenas y viceversa para ver las buenas.

In [15]:
datosCorrectos = go.Bar(
                x = correctos_x,
                y = correctos_y,
                name = "Lectura correcta",
                marker = dict (color = "#39ADFB"))

datosFallos = go.Bar(
                x = fallos_x,
                y = fallos_y,
                name = 'Lectura fallida',
                marker = dict (color = '#FF0000'))

datos = [datosCorrectos, datosFallos]

Se configura el gráfico, diciendo el título del gráfico y el título que cada eje. Por último, se pasará tanto los datos como la configuración del gráfico y se mostrará.

In [16]:
layout = go.Layout(barmode='group') #tipo de gráfico que se usará

layout = go.Layout( title = "<b>Lecturas de secuencias</b>",
                    xaxis = dict(
                        title ='Posición de la secuencia dentro del FICHERO'
                        #rangeslider = dict()#se utiliza para añadir la barra debajo 
                        ),
                    yaxis = dict(title =' Tamaño de la secuencia'))

grafico = go.Figure(data = datos, layout = layout)

plot(grafico, 
     filename = 'SalidasGraficas/GraficoDeBarras.html',
     auto_open=False)

'file://C:\\Users\\orlan\\Desktop\\TFG\\Cuadernos\\SalidasGraficas\\GraficoDeBarras.html'

In [17]:
HTML('<iframe src=SalidasGraficas/GraficoDeBarras.html width=100% height=550></iframe>')

# Gráfica para ver el porcentaje de los errores.
Se le pasarán el tamaño de los diccionarios dic_lec_buenas y dic_lec_malas, se indicará que el gráfico será de tipo Pie (gráfico de pastel).

# Visualizar las secuencias que se detectan como error
Para ver las secuencias y comprobar que corresponde con los gráficos, se muestran los datos indicando la posición de la lectura, la secuencia y las letras que más se repiten.

# Lectura para el analisis de los filtros Contenido GC y Calidades

In [18]:
for i, secuencia in enumerate(fichero_de_secuencias):
    #contenido en gc
    dic_contenido_gc[i] = (GC(secuencia.seq))
    #suma de calidades
    suma_calidades.append(np.mean(secuencia.letter_annotations["phred_quality"]))

# Contenido GC

Contenido GC o Porcentaje GC (contenido de guanina y citosina) es una característica del genoma de un organismo o de cualquier pedazo de ADN o ARN. G y C denotan guanina y citosina, respectivamente. Expresado generalmente como porcentaje, representa la cantidad de pares Guanina-Citosina en la molécula de ADN o genoma que está siendo investigado. La fracción restante de cualquier molécula de ADN contendrá bases A (adenina) y T (timina), de forma que el contenido GC da también el contenido AT.
[Leer más](https://es.wikipedia.org/wiki/Contenido_GC)

**array_gc_valores** guarda el _contenido GC_ ordenados de menor a mayor.

In [19]:
array_gc_valores = sorted(GC(rec.seq) for rec in fichero_de_secuencias)

**sec_a_quitar ** se guarda el valor obtenido de porcentaje _ inverso() que no es más que el número de secuencias que se va a quitar para la media recortada.

In [20]:
sec_a_quitar = c_externa_utile.porcentaje_inverso(D_PORC_ATIPICOS,tamanio)

**inicio** y **fin** es la posición de inicio y final que tendrá el array ->**array_gc_val_recortados** creado a partir de **array_gc_valores**

In [21]:
inicio = sec_a_quitar
fin = (tamanio-inicio)

In [22]:
array_gc_val_recortados = array_gc_valores[inicio:fin]

**med_recortada_gc** y **dt_recortada_gc** son, respectivamente, la media y la desviación típica o estándar del array **array_gc_val_recortados**

In [23]:
med_recortada_gc = np.mean(array_gc_val_recortados)
dt_recortada_gc = np.std(array_gc_val_recortados)

Se preparan los diccionarios **dic_gc_buenos** y **dic_gc_malos** en los que se guardarán el contenido gc y la posición que pasen el filtro **[ nt ± IT_CONFIANZA * σ ] ** y los que no respectivamente.
>**Nota:** nt = media recortada

In [24]:
limite_superior = med_recortada_gc + IT_CONFIANZA * dt_recortada_gc
limite_inferior = med_recortada_gc - IT_CONFIANZA * dt_recortada_gc

In [25]:
for clave, valor in dic_contenido_gc.items():
    if(valor >= limite_inferior
      and valor <= limite_superior):
        dic_gc_buenos[clave] = valor
    else:
        dic_gc_malos[clave] = valor

In [26]:
titulo_gc = "Lectura de secuencias Contenido GC"
leyenda_b_gc = "% Contenido GC bueno"
leyenda_m_gc = "% Contenido GC malo"

In [27]:
titulo_nombre_fich_gc = FICHERO_ENTRADA.split("/")[1] #extraigo el nombre del fichero

In [28]:
porc_buenos_gc = c_externa_utile.tanto_por_ciento(len(dic_gc_buenos),tamanio)

In [29]:
titulo_graf_gc ="<b> %s </b></br></br> <b>Fichero:</b> %s </br> %.2f%s </br>%.2f%s"%(titulo_gc,
                                                             titulo_nombre_fich_gc,
                                                             porc_buenos_gc,
                                                             leyenda_b_gc,
                                                             (100-porc_buenos_gc),
                                                             leyenda_m_gc)

In [30]:
grfPto=gp.GraficoDePuntos()
grafico = grfPto.crear_grafico_puntos(
                                    dic_gc_buenos,
                                    dic_gc_malos,
                                    titulo_graf_gc,
                                    'Posición de la secuencia',
                                    "Contenido GC",
                                    tamanio,
                                    med_recortada_gc,
                                    IT_CONFIANZA,
                                    limite_inferior,
                                    limite_superior
                                   )
eliminar_btn_barra  = grfPto.eliminar_btn_barra()
plot(grafico,
     filename = 'SalidasGraficas/GraficoDePuntosGC.html',
     auto_open=False,
     config= eliminar_btn_barra)
#añadir la opción image="png" para guardar la imagen en png

'file://C:\\Users\\orlan\\Desktop\\TFG\\Cuadernos\\SalidasGraficas\\GraficoDePuntosGC.html'

In [31]:
HTML('<iframe src=SalidasGraficas/GraficoDePuntosGC.html width=100% height=550></iframe>')

# Visualizar las secuencias que se detectan como error en el contenido GC

Para ver las secuencias y comprobar que corresponde con los gráficos, se muestran los datos indicando la posición de la lectura, la secuencia y el contenido GC.

# Calidad de los fallidos
Se miran las calidades de todas las lecturas y a partir de ellas se sacarán las peores, para su posterior comparación con las ya obtenidas anteriormente.
>Procedimiento para sacar los posibles fallos según las calidades

>* Se obtiene la media y la desviación estándar de las calidades de todas las lecturas.
>* Se aplica _la formula **[ x̄ ± 1.64 * σ ]**_ para sacar el límite superior e inferior
>* Se busca cuales lecturas están dentro de estos límites y cuáles no.


In [32]:
array_cal_valores = sorted((np.mean(rec.letter_annotations["phred_quality"])) for rec in fichero_de_secuencias)

In [33]:
array_cal_valores = array_cal_valores[inicio:fin]

In [34]:
med_calidad_rec = np.mean(array_cal_valores)
dt_calidad_rec = np.std(array_cal_valores)

In [35]:
limite_inferior_cal = med_calidad_rec - IT_CONFIANZA * dt_calidad_rec

Separación de las calidades "buenas" y "malas"

In [36]:
for  i, sec_cal in enumerate(fichero_de_secuencias):   
    med_calidad = np.mean(sec_cal.letter_annotations["phred_quality"])
    if(med_calidad > limite_inferior_cal):
        dic_cal_buenas[i] = med_calidad
    else:
        dic_cal_malas[i] = med_calidad

Preparación de los datos para posteriormente mostrarlos en una gráfica.

In [37]:
titulo_c = "Lectura de secuencias Calidades"
leyenda_b_c = "% Calidades buenas"
leyenda_m_c = "% Calidades malas"

In [38]:
titulo_nombre_fich_c = FICHERO_ENTRADA.split("/")[1] #extraigo el nombre del fichero

In [39]:
porc_buenos_c = c_externa_utile.tanto_por_ciento(len(dic_cal_buenas),tamanio)

In [40]:
titulo_graf_c ="<b> %s </b></br></br> <b>Fichero:</b> %s </br> %.2f%s </br>%.2f%s"%(titulo_c,
                                                             titulo_nombre_fich_c,
                                                             porc_buenos_c,
                                                             leyenda_b_c,
                                                             (100-porc_buenos_c),
                                                             leyenda_m_c)

In [41]:
grfPtoC = gp.GraficoDePuntos()
graficoC = grfPtoC.crear_grafico_puntos(dic_cal_buenas,
                                        dic_cal_malas,
                                        titulo_graf_c,
                                        'Posición de la secuencia',
                                        'Media Calidad',
                                        tamanio,
                                        med_calidad_rec,
                                        IT_CONFIANZA,
                                        limite_inferior_cal
                                       )
eliminar_btn_barra  = grfPtoC.eliminar_btn_barra()
plot(graficoC, 
     filename = 'SalidasGraficas/GraficoDePuntosCalidades.html',
     auto_open=False,
     config= eliminar_btn_barra
    )

'file://C:\\Users\\orlan\\Desktop\\TFG\\Cuadernos\\SalidasGraficas\\GraficoDePuntosCalidades.html'

In [42]:
HTML('<iframe src=SalidasGraficas/GraficoDePuntosCalidades.html width=100% height=550></iframe>')

# Visualizar las secuencias que se detectan como error en las calidades

Para ver las secuencias y comprobar que corresponde con los gráficos, se muestran los datos indicando la posición de la lectura, la secuencia y la media de las calidades.


# Tabla de comparaciones
A continuación se mostrará una tabla con las posiciones de los posibles errores de cada uno de los métodos (GC, calidades y ~~permutaciones~~-)

In [43]:
array_pos_malos_gc = c_arrays_utile.obtener_claves_dic(dic_gc_malos)
array_pos_malos_calidad =  c_arrays_utile.obtener_claves_dic(dic_cal_malas)

In [44]:
arrays_pos = c_arrays_utile.igualar_arrays(array_pos_malos_gc, array_pos_malos_calidad)

In [45]:
array_pos_malos_perm = c_arrays_utile.obtener_claves_dic(dic_lec_malas)
array_p = c_arrays_utile.igualar_arrays(array_pos_malos_gc,array_pos_malos_perm)

>**NOTA:**Este método, *colores_tabla* se utiliza para llamar a otro, el otro método llamado igual pero en una clase externa recibe 3 parámetros pero **applypmap** no permite hacer una llamada a un método que reciba más de un parámetro.

In [46]:
def colores_tabla(val):
    return c_psd.colores_tabla(val, 
                               arrays_pos[0],
                               arrays_pos[1])

In [47]:
dic_pandas = {'Contenido GC':arrays_pos[0], 
              'Calidades':arrays_pos[1]}
dtaF = pd.DataFrame(dic_pandas)

In [48]:
tabla = dtaF.style.applymap(colores_tabla)
tabla

,Calidades,Contenido GC
0,-,0
1,1,-
2,11,11
3,12,12
4,13,13
5,-,16
6,17,17
7,19,-
8,20,-
9,22,22


In [49]:
res_coincide = c_arrays_utile.coicidencias(arrays_pos[0],arrays_pos[1])
c_psd.printmd('* Total de errores obtenidos de la distribución en **Calidad: %i lecturas.**'%(len(array_pos_malos_calidad)))
c_psd.printmd('* Total de errores obtenidos de la distribución con el **Contenido GC: %i lecturas.**'%len(array_pos_malos_gc))
c_psd.printmd('* En total, de los fallos en Calidades y fallos en Contenido GC, **%i coiciden entre ellos.**'%len(res_coincide))
c_psd.printmd('* **Total en porcentaje:%.2f%s del total de las lecturas.**'%((c_externa_utile.tanto_por_ciento(len(res_coincide),
                                                                                                              tamanio)),"%"))

* Total de errores obtenidos de la distribución en **Calidad: 206 lecturas.**

* Total de errores obtenidos de la distribución con el **Contenido GC: 178 lecturas.**

* En total, de los fallos en Calidades y fallos en Contenido GC, **156 coiciden entre ellos.**

* **Total en porcentaje:13.32% del total de las lecturas.**

# Exportar resultados a ficheros FastQ

A continuación, se guardarán los resultados obtenidos en la siguiente jerarquía de directorios:
* **Método de dos letras más repetidas**
    * Salidas/DosMasRepetidas/
* **Método de Contenido GC**
    * Salidas/ContenidoGC/
* **Método de Calidades**
    * Salidas/Calidades/
* **Coincidencias en los filtros de calidad y contendio GC**
    * Salidas/Coincidencias/
    
>**NOTA:** Dentro de cada uno de estos directorios se encontrará un fichero FastQ con las lecturas fallidas y otro con las lecturas buenas.

**1.Primero se extraen las posiciones de las lecturas, tanto buenas como fallidas, de los tres métodos.**

In [50]:
a_dos_rep_malas = c_arrays_utile.obtener_claves_dic(dic_lec_malas)
a_dos_rep_buenas = c_arrays_utile.obtener_claves_dic(dic_lec_buenas)

In [51]:
a_gc_malos = c_arrays_utile.obtener_claves_dic(dic_gc_malos)
a_gc_buenos = c_arrays_utile.obtener_claves_dic(dic_gc_buenos)

In [52]:
a_cal_malas = c_arrays_utile.obtener_claves_dic(dic_cal_malas)
a_cal_buenas = c_arrays_utile.obtener_claves_dic(dic_cal_buenas)

**2.Se extraen las secuencias completas que se encuentran en las posiciones extraídas anteriormente.**

In [53]:
a_sec_dos_rep_malas= c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_dos_rep_malas)
a_sec_dos_rep_buenas= c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_dos_rep_buenas)

In [54]:
a_sec_gc_malos = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_gc_malos)
a_sec_gc_buenos = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_gc_buenos)

In [55]:
a_sec_cal_malas = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_cal_malas)
a_sec_cal_buenas = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,a_cal_buenas)

In [56]:
a_coincide = c_arrays_utile.obtener_array_sec(fichero_de_secuencias,res_coincide)

**3.Se procede a guardar las secuencias en los distintos ficheros de salidas.**

In [57]:
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_DOS_MAS_REP+FICHERO_SALIDA_MALAS,
                                    FORMATO,
                                    a_sec_dos_rep_malas)
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_DOS_MAS_REP+FICHERO_SALIDA_BUENAS,
                                    FORMATO,
                                    a_sec_dos_rep_buenas)

In [58]:
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_GC+FICHERO_SALIDA_MALAS,
                                    FORMATO,
                                    a_sec_gc_malos)
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_GC+FICHERO_SALIDA_BUENAS,
                                    FORMATO,
                                    a_sec_gc_buenos)

In [59]:
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_CALIDAD+FICHERO_SALIDA_MALAS,
                                    FORMATO,
                                    a_sec_cal_malas)
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_CALIDAD+FICHERO_SALIDA_BUENAS,
                                    FORMATO,
                                    a_sec_cal_buenas)

In [60]:
a_no_coicide = []
for i,sec in enumerate(fichero_de_secuencias):
    if (i not in res_coincide):
        a_no_coicide.append(sec)

In [61]:
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_COIN+FICHERO_SALIDA_MALAS,
                                    FORMATO,
                                    a_coincide)
c_externa_utile.guardar_sec_fichero(PATH_SALIDA_COIN+FICHERO_SALIDA_BUENAS,
                                    FORMATO,
                                    a_no_coicide)

In [62]:
print(PATH_SALIDA_COIN+FICHERO_SALIDA_MALAS)

Salidas/Coincidencia/Lecturas_malas.fastq


# Comprobación con BLAST

Ahora se pasará a comprobar que, de entre las secuencias que coinciden como error tanto en un filtrado como en el otro, la base de Datos [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi). Puesto que la api con la que cuenta Biopython no funciona como debería, se pasó a realizar una "propia" con la cual utilizando [Web Scraping](https://es.wikipedia.org/wiki/Web_scraping) y en combinación con la propia API web de BLAST, haciendo una combinación de estas se pudo realizar las peticiones a BLAST.

In [63]:
c_externa_utile.convertidor_formatos(FICHERO_ENTRADA_BLAST,FORMATO_ENTRADA_BLAST,FICHERO_SALIDA_BLAST,FORMATO_SALIDA_BLAST)

In [64]:
#Lectura del fichero .fasta
fich_fasta  = [rec for rec in SeqIO.parse(FICHERO_SALIDA_BLAST, FORMATO_SALIDA_BLAST)]

In [65]:
for i, sec in enumerate(fich_fasta):
    sec_formateada = sec.seq
    if(len(sec_formateada) > 8000):
        sec_formateada = sec_formateada[0:8000]

    peticion_put.construir_peticion(sec_formateada,
                                  BASE_DE_DATOS_BLAST,
                                  PROGRAMA_BLAST)
    peticion_put.realizar_peticion()
    dic_rids[peticion_put.rid] = sec_formateada
    time.sleep(10) # tiempo para evitar posible bloqueo por parte del servidor de BLAST

> **Nota:** Tras realizar distintas pruebas se pudo llegar a la conclusión de que, para realizar las consultas, la secuencia no debe sobrepasar los 8100 caracteres, para no llegar hasta dicho extremo las secuencias la limitaremos a 8000 para evitar posibles errores. Además,g se añadió un tiempo de espera de 11 segundos para evitar que el servidor de BLAST detecte las peticiones como ataque y bloquee el acceso a esta desde la IP donde se está realizando las peticiones.

In [66]:
for rid, sec in dic_rids.items():
    peticion_get.construir_peticion(rid)
    array_resultados_blast.append(peticion_get.realizar_peticion())

In [67]:
porc_false = array_resultados_blast.count(False)
porc_true = array_resultados_blast.count(True)
c_psd.printmd("* **Errores detectados que NO encontró coincidencias en BLAST:** %i"%porc_false)
c_psd.printmd("* **Errores detectados que SI encontró coincidencias en BLAST:** %i"%porc_true)

porcentaje = (c_externa_utile.tanto_por_ciento(porc_true,
                                               len(array_resultados_blast)))
              
c_psd.printmd("* **Porcentaje de errores:**%.2f (Porcentaje que **NO** obtuvo evidencia en BLAST)"%(100 - float(porcentaje)))
c_psd.printmd("* **Porcentaje de acierto:**%.2f (Porcentaje que **SI** obtuvo evidencia en BLAST)"%porcentaje)

* **Errores detectados que NO encontró coincidencias en BLAST:** 152

* **Errores detectados que SI encontró coincidencias en BLAST:** 4

* **Porcentaje de errores:**97.44 (Porcentaje que **NO** obtuvo evidencia en BLAST)

* **Porcentaje de acierto:**2.56 (Porcentaje que **SI** obtuvo evidencia en BLAST)